<a href="https://colab.research.google.com/github/mertege/FMCW-Data-Classification-/blob/main/Range_Doppler_and_Spectrogram_CNN_compare_slow_and_pocket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Burada range-doppler ve spectrogram dataları CNN için birlikte kullanıldı. Slow ve slow with pocket birlikte karşılaştırıldı.
# 5 defa run edildi ve değerler aşağıda paylaşıldı.
# Mean test accuracy is 0.54, mean test f1 score is 0.44, max test accuracy is 0.54, max test f1 score is 0.44, 
# Min test accuracy is 0.54, min test f1 score is 0.44, std of test accuracy is 0.00, std of test f1 score is 0.00
# Time elapsed through all process: 1043.46, sec

In [ ]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get Range-Doppler data from

range_doppler_slow_resized = scipy.io.loadmat('/content/drive/MyDrive/range_doppler_slow_resized.mat')
range_doppler_slow_resized = range_doppler_slow_resized['range_doppler_slow_resized']
range_doppler_slow_resized = np.transpose(range_doppler_slow_resized, (2, 0, 1))
range_doppler_slow_label = scipy.io.loadmat('/content/drive/MyDrive/range_doppler_slow_label.mat')
range_doppler_slow_label = range_doppler_slow_label['range_doppler_slow_label']  

range_doppler_slow_pocket_resized = scipy.io.loadmat('/content/drive/MyDrive/range_doppler_slow_pocket_resized.mat')
range_doppler_slow_pocket_resized = range_doppler_slow_pocket_resized['range_doppler_slow_pocket_resized']
range_doppler_slow_pocket_resized = np.transpose(range_doppler_slow_pocket_resized, (2, 0, 1))
range_doppler_pocket_label = scipy.io.loadmat('/content/drive/MyDrive/range_doppler_pocket_label.mat')
range_doppler_pocket_label = range_doppler_pocket_label['range_doppler_pocket_label']  
# Get Spectrogram data from

spectrogram_slow_resized = scipy.io.loadmat('/content/drive/MyDrive/spectrogram_slow_resized.mat')
spectrogram_slow_resized = spectrogram_slow_resized['spectrogram_slow_resized']
spectrogram_slow_resized = np.transpose(spectrogram_slow_resized, (2, 0, 1))
spectrogram_slow_label = scipy.io.loadmat('/content/drive/MyDrive/spectrogram_slow_label.mat')
spectrogram_slow_label = spectrogram_slow_label['spectrogram_slow_label']  

spectrogram_slow_pocket_resized = scipy.io.loadmat('/content/drive/MyDrive/spectrogram_slow_pocket_resized.mat')
spectrogram_slow_pocket_resized = spectrogram_slow_pocket_resized['spectrogram_slow_pocket_resized']
spectrogram_slow_pocket_resized = np.transpose(spectrogram_slow_pocket_resized, (2, 0, 1))
spectrogram_slow_pocket_label = scipy.io.loadmat('/content/drive/MyDrive/spectrogram_slow_pocket_label.mat')
spectrogram_slow_pocket_label = spectrogram_slow_pocket_label['spectrogram_slow_pocket_label']  

In [ ]:
# Concat range-doppler data
range_doppler_concat = np.concatenate((range_doppler_slow_pocket_resized, range_doppler_slow_resized),axis=0)
range_doppler_concat = range_doppler_concat[:,:,:,np.newaxis] 
range_doppler_concat_label = np.zeros((range_doppler_concat.shape[0],1))
range_doppler_concat_label[:range_doppler_slow_pocket_resized.shape[0],:] = 1 # pocket = 1, slow = 0
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]
# Concat range-doppler data
spectrogram_concat = np.concatenate((spectrogram_slow_pocket_resized, spectrogram_slow_resized),axis=0)
spectrogram_concat = spectrogram_concat[:,:,:,np.newaxis] 
spectrogram_concat_label = np.zeros((spectrogram_concat.shape[0],1))
spectrogram_concat_label[:spectrogram_slow_pocket_resized.shape[0],:] = 1 # pocket = 1, slow = 0
# Shuffle concat range doppler
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]  

In [ ]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.1),
])
def split_and_augmentation_of_training(range_doppler_concat_shuffle_train,range_doppler_concat_label_shuffle_train, randomlist_for_validation_indx, augmentation_enable):
  # ---------------- Parameters ----------------
  repeat_of_augmentation = 5
  size_of_validation = 16

  # Split validation
  if bool(randomlist_for_validation_indx) == False:
    randomlist_for_validation_indx = random.sample(range(0, range_doppler_concat_shuffle_train.shape[0]), size_of_validation) # split validation data
    randomlist_for_train_indx = np.delete(range(0, range_doppler_concat_shuffle_train.shape[0]), randomlist_for_validation_indx) # split training data
  else:
    randomlist_for_validation_indx = randomlist_for_validation_indx # split validation data
    randomlist_for_train_indx = np.delete(range(0, range_doppler_concat_shuffle_train.shape[0]), randomlist_for_validation_indx) # split training data

  validation_spectrograms = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  validation_labels = range_doppler_concat_label_shuffle_train[randomlist_for_validation_indx,:]
  # get training data
  training_spectrograms = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  training_labels = range_doppler_concat_label_shuffle_train[randomlist_for_train_indx,:]
  # get slow and pocket indexes of training data
  slow_indexes = np.where(training_labels == 0)[0]
  pocket_indexes = np.delete(range(0, training_labels.shape[0]), slow_indexes)  

  slow_spectrograms_train = training_spectrograms[slow_indexes,:,:,:]
  size_of_samples_slow = slow_spectrograms_train.shape[0]

  pocket_spectrograms_train = training_spectrograms[pocket_indexes,:,:,:]  
  size_of_samples_pocket = pocket_spectrograms_train.shape[0]

  # -- Concat pocket and Slow Data at Validation Dataset -- 
  # validation_spectrograms = np.concatenate((pocket_spectrograms_val,slow_spectrograms_val),axis=0)
  # validation_labels = np.concatenate((pocket_label_val,slow_label_val),axis=0)
  if augmentation_enable == True: 
    # ---------------- Augmente Train Data for pocket ----------------
    augmented_image_pocket = np.zeros((size_of_samples_pocket*repeat_of_augmentation,pocket_spectrograms_train.shape[1],pocket_spectrograms_train.shape[2],1))
    spectrograms_pocket_label = np.ones((size_of_samples_pocket*(repeat_of_augmentation+1),1))
    for jj in range(repeat_of_augmentation):
      for ii in range(size_of_samples_pocket):
        augmented_image_pocket[size_of_samples_pocket*jj+ii,:,:,:] = data_augmentation(pocket_spectrograms_train[ii,:,:,:])
    augmented_image_pocket = np.concatenate((augmented_image_pocket,pocket_spectrograms_train),axis=0)   

    # ---------------- Augmente Train Data for Slow ----------------
    augmented_image_slow = np.zeros((size_of_samples_slow*repeat_of_augmentation,slow_spectrograms_train.shape[1],slow_spectrograms_train.shape[2],1))
    spectrograms_slow_label = np.zeros((size_of_samples_slow*(repeat_of_augmentation+1),1))
    if repeat_of_augmentation == 0:
      augmented_image_slow = slow_spectrograms_train
    else:
      for kk in range(repeat_of_augmentation):
        for ii in range(size_of_samples_slow):
          augmented_image_slow[size_of_samples_slow*kk+ii,:,:,:] = data_augmentation(slow_spectrograms_train[ii,:,:,:])
      augmented_image_slow = np.concatenate((augmented_image_slow,slow_spectrograms_train),axis=0)    
  else:
    augmented_image_pocket = pocket_spectrograms_train
    augmented_image_slow = slow_spectrograms_train
    spectrograms_pocket_label = np.ones((size_of_samples_pocket,1))
    spectrograms_slow_label = np.zeros((size_of_samples_slow,1))
  return (augmented_image_pocket,augmented_image_slow,spectrograms_pocket_label,spectrograms_slow_label,validation_spectrograms,validation_labels, randomlist_for_validation_indx)


def normalize_inputs(range_doppler_concat_shuffle_test, validation_spectrograms, augmented_image, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle_test)
    range_doppler_concat_shuffle_test = layer(range_doppler_concat_shuffle_test)
    layer = Normalization(axis=None)
    layer.adapt(validation_spectrograms)
    validation_spectrograms = layer(validation_spectrograms)
    layer = Normalization(axis=None)
    layer.adapt(augmented_image)
    augmented_image = layer(augmented_image)
  else:
    (range_doppler_concat_shuffle_test, validation_spectrograms, augmented_image) = (range_doppler_concat_shuffle_test, validation_spectrograms, augmented_image)
  return(range_doppler_concat_shuffle_test, validation_spectrograms, augmented_image)

In [ ]:
t = time.time()
# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 32
dropout_prob_cnn = 0.1
dropout_prob_dense = 0.1
number_of_repeat = 1
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  for train, test in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):   

    # ---------------- Range-Doppler Data ----------------
    randomlist_for_test_indx = test
    randomlist_for_train_indx = train
    # test data
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    range_doppler_concat_label_shuffle_test = range_doppler_concat_label_shuffle[randomlist_for_test_indx,:]
    #train data
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    range_doppler_concat_label_shuffle_train = range_doppler_concat_label_shuffle[randomlist_for_train_indx,:]
      # ---------------- Split labels to equal them during augmentation for Validation ----------------
    randomlist_for_validation_indx = []
    (range_doppler_augmented_image_pocket,range_doppler_augmented_image_slow,range_doppler_pocket_label,range_doppler_slow_label,validation_range_doppler,\
     range_doppler_validation_labels, randomlist_for_validation_indx)  = split_and_augmentation_of_training(range_doppler_concat_shuffle_train,range_doppler_concat_label_shuffle_train,randomlist_for_validation_indx, augmentation_enable)
    
    # ---------------- Concat Validation and Slow ----------------
    range_doppler_augmented_image = np.concatenate((range_doppler_augmented_image_pocket,range_doppler_augmented_image_slow),axis=0)
    range_doppler_concat_label_shuffle_concat = np.concatenate((range_doppler_pocket_label,range_doppler_slow_label),axis=0)

    range_doppler_augmented_image = np.concatenate((range_doppler_augmented_image_pocket,range_doppler_augmented_image_slow),axis=0)
    range_doppler_concat_label_shuffle_concat = np.concatenate((range_doppler_pocket_label,range_doppler_slow_label),axis=0)   

    # ---------------- Spectrogram Data ----------------
    # test data
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_validation_indx,:]
    #train data
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
      # ---------------- Split labels to equal them during augmentation for Validation ----------------
    (spectrogram_augmented_image_pocket,spectrogram_augmented_image_slow,spectrogram_pocket_label,spectrogram_slow_label,validation_spectrogram,\
     spectrogram_validation_labels, randomlist_for_validation_indx)  = split_and_augmentation_of_training(spectrogram_concat_shuffle_train,spectrogram_concat_label_shuffle_train,randomlist_for_validation_indx, augmentation_enable)
    # ---------------- Concat Validation and Slow ----------------
    spectrogram_augmented_image = np.concatenate((spectrogram_augmented_image_pocket,spectrogram_augmented_image_slow),axis=0)
    spectrogram_concat_label_shuffle_concat = np.concatenate((spectrogram_pocket_label,spectrogram_slow_label),axis=0)


    # ---------------- Normalization ----------------
    (range_doppler_concat_shuffle_test, validation_range_doppler, range_doppler_augmented_image) = normalize_inputs(range_doppler_concat_shuffle_test,\
                                                                                                     validation_range_doppler, range_doppler_augmented_image, normalize_inputs_enable)   
    (spectrogram_concat_shuffle_test, validation_spectrogram, spectrogram_augmented_image) = normalize_inputs(spectrogram_concat_shuffle_test,\
                                                                                                     validation_spectrogram, spectrogram_augmented_image, normalize_inputs_enable)
    # ---------------- Neural Network Architecture ----------------


    def encoder_for_range_doppler(input_shape):
      input = Input(shape=input_shape)
      x = Conv2D(8, (3, 3),padding='same',kernel_initializer="glorot_normal")(input)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(32, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Flatten()(x)
      return Model(input, x)

    def encoder_for_spectrogram(input_shape):
      input = Input(shape=input_shape)
      x = Conv2D(8, (3, 3),padding='same',kernel_initializer="glorot_normal")(input)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(32, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Flatten()(x)
      return Model(input, x)

    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(128, activation="relu")(input)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(dense_size, activation="relu")(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(1, activation="sigmoid")(x)
      return Model(input, x)

    input_shape = range_doppler_concat_shuffle.shape[1:]
    base_network_range_doppler = encoder_for_range_doppler(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_range_doppler(range_doppler_input)

    input_shape = spectrogram_concat_shuffle.shape[1:]
    base_network_spectrogram = encoder_for_spectrogram(input_shape) 
    spectrogram_input  = Input(shape=input_shape)
    processed_spectrogram  = base_network_spectrogram(spectrogram_input)

    concat_layer = Concatenate()([processed_range_doppler, processed_spectrogram])

    base_decoder_network = decoder_for_concat((concat_layer.shape[1]))
    out = base_decoder_network(concat_layer)

    model = Model(inputs=[range_doppler_input, spectrogram_input], outputs=[out])

   
    print(model.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=35, verbose=0,restore_best_weights=True, mode='min')
    # earlyStopping = EarlyStopping(monitor='val_accuracy', patience=15, verbose=0,restore_best_weights=True, mode='max')
    history = model.fit((range_doppler_augmented_image,spectrogram_augmented_image),(range_doppler_concat_label_shuffle_concat),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((validation_range_doppler,validation_spectrogram) , (range_doppler_validation_labels)))
    tf.keras.models.load_model
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test,spectrogram_concat_shuffle_test],\
                                               [range_doppler_concat_label_shuffle_test],
                  batch_size=batch_size)
    # ---------------- Get Test Results ----------------
    y_test_predicted = model.predict((range_doppler_concat_shuffle_test,spectrogram_concat_shuffle_test), batch_size=batch_size)
    # ----- Binarize y_test_predicted values -----
    y_test_predicted_binary = np.zeros(y_test_predicted.size)
    for ii in range(y_test_predicted.size):
      if y_test_predicted[ii] < 0.5:
        y_test_predicted_binary[ii] = 0
      else:
        y_test_predicted_binary[ii] = 1
    
    test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(range_doppler_concat_label_shuffle_test, y_test_predicted_binary, average='macro')

    test_accuracy_per_fold.append(test_accuracy)
    f1_score_per_fold.append(test_f1_score)

  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  f1_score_per_run.append(sum(f1_score_per_fold)/num_folds)
print(f'Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.2f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.2f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.2f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.2f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.2f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.2f}".format(np.std(f1_score_per_run, axis=0))}')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 52, 5120, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_19 (InputLayer)          [(None, 52, 423, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 model_12 (Functional)          (None, 122880)       6112        ['input_17[0][0]']               
                                                                                           

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 52, 5120, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_24 (InputLayer)          [(None, 52, 423, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 model_16 (Functional)          (None, 122880)       6112        ['input_22[0][0]']               
                                                                                           

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_23"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 52, 5120, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_29 (InputLayer)          [(None, 52, 423, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 model_20 (Functional)          (None, 122880)       6112        ['input_27[0][0]']               
                                                                                           

In [ ]:
print(f'Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.2f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.2f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.2f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.2f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.2f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.2f}".format(np.std(f1_score_per_run, axis=0))}')
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

Mean test accuracy is 0.54, mean test f1 score is 0.44, max test accuracy is 0.54, max test f1 score is 0.44, min test accuracy is 0.54, min test f1 score is 0.44, std of test accuracy is 0.00, std of test f1 score is 0.00
Time elapsed through all process: 1043.46, sec


In [ ]:
print(y_test_predicted)

[[0.44536555]
 [0.4577653 ]
 [0.46605754]
 [0.47095448]
 [0.4970678 ]
 [0.48563138]
 [0.49044713]
 [0.4932826 ]
 [0.4489855 ]
 [0.4195289 ]
 [0.45070305]
 [0.4541195 ]
 [0.4768964 ]
 [0.4493063 ]
 [0.4636053 ]
 [0.45973113]
 [0.40939012]
 [0.43089116]
 [0.4590946 ]
 [0.4588199 ]
 [0.49398267]
 [0.46205243]
 [0.44504672]]
